In [19]:
import requests
import pandas as pd

url = 'https://restapi.amap.com/v3/place/text'

#设置参数
parameters = {'key':'c4c6******************1b3f',  #填写高德地图API的key
              #'keywords':'金融保险',
              'types':'160000',                    #金融保险服务
              'city':'430100',                     #长沙市
              'citylimit':'true',
              'offset':20,
              'page':1
             }

#获取高德地图的数据
def getGaodeMapInfo(url = url , parameters = parameters):
    gaode = requests.get(url,parameters)
    #获取返回的json数据
    result = gaode.json()
    return result

#获取首次请求数据时得到的最大页数
def getMaxPage(result):
    pages = int(result['count']) / 20 + 1
    return pages

result = getGaodeMapInfo()

pages = getMaxPage(result)

pages

46.0

In [40]:
poisInfo = None

In [41]:
n = range(1,int(pages))

#创建一张空的目标表格
poisInfo = pd.DataFrame(columns=['name','pname','cityname','adname','address','type','location'])

#循环访问每一页
for page in n :
    parameters['page'] = page
    result = getGaodeMapInfo(url = url , parameters = parameters)
    
    #将数据转换成DataFrame表格形式
    pois = pd.DataFrame(result['pois'])
    
    #选取必要的字段
    info = pois.loc[:,['name','pname','cityname','adname','address','type','location']]
    
    #合并表
    poisInfo = poisInfo.append(info,ignore_index = True)
        
    
len(poisInfo)

900

In [51]:
#清洗数据
#清除为空的数据
poisInfo_cleaned = poisInfo.dropna(how = 'all')

len(poisInfo_cleaned)

900

In [52]:
poisInfo_cleaned.head(10)

,name,pname,cityname,adname,address,type,location
0,长沙银行(湘府路支行),湖南省,长沙市,雨花区,湘府东路258号双塔国际广场1层,金融保险服务;银行;银行,"113.024542,28.111527"
1,长沙银行(高升路支行),湖南省,长沙市,雨花区,高升路268号,金融保险服务;银行;银行,"113.006122,28.096131"
2,长沙银行(高桥支行),湖南省,长沙市,雨花区,[],金融保险服务;银行;银行,"113.029341,28.181563"
3,方正证券,湖南省,长沙市,雨花区,韶山南路777号4层,金融保险服务;证券公司;证券公司,"113.005319,28.127533"
4,长沙银行(银德支行),湖南省,长沙市,天心区,芙蓉中路三段266号弘林国际大厦,金融保险服务;银行;银行,"112.985314,28.165562"
5,华融湘江银行(长沙分行),湖南省,长沙市,雨花区,湘府东路208号万境财智中心,金融保险服务;银行;银行,"113.028316,28.110804"
6,中国农业银行(高云支行),湖南省,长沙市,天心区,暮云村高云村长沙理工大学新校区行健轩2栋,金融保险服务;银行;中国农业银行,"113.010022,28.070508"
7,长沙银行(南城支行),湖南省,长沙市,天心区,湘府西路31号鑫远国际大厦1层,金融保险服务;银行;银行,"112.977875,28.111662"
8,长沙银行(汇融支行),湖南省,长沙市,天心区,芙蓉中路三段558号现代空间大厦,金融保险服务;银行;银行,"112.984839,28.149473"
9,人保财险(湖南省分公司),湖南省,长沙市,开福区,湘雅路167号,金融保险服务;金融保险服务机构;金融保险机构,"112.981554,28.213350"


In [47]:
#导出csv文件

import os

path = os.getcwd()

poisInfo_cleaned.to_csv(path + '\\data\\pois_changsha.csv',index = 0,encoding='utf-8')
